# KPIs

En este apartado realizaremos la creacion de los dataframes necesarios para el calculo de cada KPIs a apartir del dataframe orginal que es el df_siniestros

## 1. Importamos librerias necesarias

In [53]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## 2. Importamos los datos originales

In [54]:
df_siniestros = pd.read_csv("Datasets\df_siniestros.csv")

# KPI 1
### Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

## 1. Creamos una nueva variable "Semestre" que basicamente divide cada año en dos partes

In [55]:
df_siniestros["Semestre"] = df_siniestros["Mes"].apply(lambda x: 1 if x <= 6 else 2)

## 2. Una vez que tenemos la columna Semestres realizamos la agrupacion por Año y Semestre para poder sumar la cantidad de homicidios que corresponde a cada uno

In [56]:
df_agrupado_semestral = df_siniestros.groupby(["Año","Semestre"]).size().reset_index(name="Homicidios")

## 3. Calculamos una nueva variable Homicidios Semestre Anterior donde se calcula para cada semestre anterior la cantidad de homicidios

In [59]:
df_agrupado_semestral["SemestreAnterior"] = df_agrupado_semestral["Homicidios"].shift(periods=1, fill_value=0)

## 4. Calculamos la tasa de homicidios para cada semestre cada 100000 habitantes

In [60]:
df_agrupado_semestral["TasaHomicidios"] = (df_agrupado_semestral["Homicidios"]/3120612) * 100000
df_agrupado_semestral["TasaSemestreAnterior"] = (df_agrupado_semestral["SemestreAnterior"]/3120612) * 100000

## 5. Creamos una nueva variable: "Diferencia Porcentual"

In [61]:
df_agrupado_semestral["Diferencia Porcentual"] = (df_agrupado_semestral["TasaHomicidios"] - df_agrupado_semestral["TasaSemestreAnterior"]) / df_agrupado_semestral["TasaSemestreAnterior"] * 100

## 6. Cargamos el nuevo dataframe

In [62]:
df_agrupado_semestral_csv = "Datasets/df_kpi1.csv"
df_agrupado_semestral.to_csv(df_agrupado_semestral_csv, index=False)

# KPI 2
### Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

## 1. Filtramos y seleccionamos aquellos registros donde la victima es motociclista

In [63]:
df_kpi2 = df_siniestros[(df_siniestros["Victima"] == "MOTO")]

## 2. Agrupamos por Año y contamos la cantidad de homicidios

In [64]:
df_kpi2 = df_kpi2.groupby(["Año"]).size().reset_index(name="Homicidios")

## 3. Creacion de la nueva variable Año anterior donde se indica la cantidad de homicidios ocurridos el año anterior 

In [65]:
df_kpi2["Año_Anterior"] = df_kpi2["Homicidios"].shift(periods=1, fill_value=0)

## 4. Obtenemos la tasa de homicidios cada 100000 habitantes teniendo en cuenta que la poblacion de Cuidad Autonoma de Buenos Aires al censo 2022 es de 3,120,612

In [66]:
poblacion_caba = 3120612
df_kpi2["Tasa_Homicidios"] = (df_kpi2["Homicidios"] / poblacion_caba) * 100000
df_kpi2["Tasa_Año_Anterior"] = (df_kpi2["Año_Anterior"] / poblacion_caba) * 100000

## 5. Creacion de la variable Variacion Porcentual

In [67]:
df_kpi2["Variacion"] = ((df_kpi2["Tasa_Homicidios"] - df_kpi2["Tasa_Año_Anterior"]) / df_kpi2["Tasa_Año_Anterior"]) * 100
df_kpi2["Variacion"] = df_kpi2["Variacion"].replace([np.inf, -np.inf], np.nan)
df_kpi2

,Año,Homicidios,Año_Anterior,Tasa_Homicidios,Tasa_Año_Anterior,Variacion
0,2016,63,0,2.018835,0.000000,NaN
1,2017,54,63,1.730430,2.018835,-14.285714
2,2018,57,54,1.826565,1.730430,5.555556
3,2019,50,57,1.602250,1.826565,-12.280702
4,2020,27,50,0.865215,1.602250,-46.000000
5,2021,45,27,1.442025,0.865215,66.666667


## 6. Cargamos el nuevo dataframe

In [68]:
df_kpi2_csv = "Datasets/df_kpi2.csv"
df_kpi2.to_csv(df_kpi2_csv, index=False)